# Exercise: Create your own AI image generator

In this exercise, you will create your own AI image generator. You will take the famous MNIST dataset and train a Variational Auto Encoder that will generate new images of handwritten digits.

Let's get started!

In [ ]:
# https://download.pytorch.org/whl/torch_stable.html 
! pip install torch torchvision matplotlib > /dev/null

In [ ]:
# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import matplotlib.pyplot as plt

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)



In [ ]:
# Define a VAE
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)

        self.z_dim = z_dim
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=4)

# set device

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')


In [ ]:
vae

In [ ]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    # Reconstruction error. To measure how well we have reconstructed the input image.
    # See https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html#torch.nn.BCELoss
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # KL divergence. To measure how closely the latent variables match a unit Gaussian distribution.
    # See the original paper ("Auto-Encoding Variational Bayes")[https://arxiv.org/abs/1312.6114]
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    return BCE + KLD # total loss

In [ ]:
def train(epoch):
    vae.train() # set model to training mode
    train_loss = 0

    # iterate over the training data
    for batch_idx, (data, _) in enumerate(train_loader):

        data = data.to(device) # move data to device
        optimizer.zero_grad() # clear the gradients of all optimized variables
        
        output_images, mu, log_var = vae(data) # forward pass
        loss = loss_function(output_images, data, mu, log_var) # calculate loss
        
        loss.backward() # backward pass
        train_loss += loss.item() # update running training loss
        optimizer.step() # perform a single optimization step (parameter update)
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} '
                  f'[{batch_idx * len(data)}/{len(train_loader.dataset)} {100. * batch_idx / len(train_loader):.0f}] '
                  f'Loss: {loss.item() / len(data):.6f}'
                  )
    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')

In [ ]:
# def test():
#     vae.eval()
#     test_loss= 0
#     with torch.no_grad():
#         for data, _ in test_loader:
#             # data = data.cuda()
#             recon, mu, log_var = vae(data)
            
#             # sum up batch loss
#             test_loss += loss_function(recon, data, mu, log_var).item()
        
#     test_loss /= len(test_loader.dataset)
#     print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
! rm -rf gen_mnist_img && mkdir -p gen_mnist_img

z = torch.randn(16, vae.z_dim).to(device)

def write_gen_mnist_img(z, vae, epoch):
    with torch.no_grad():
        z_decoded = vae.decoder(z).to(device)

    filename = f'./gen_mnist_img/epoch_{epoch}.png'   
    print(f"Writing {filename}")
    save_image(z_decoded.view(16, 1, 28, 28), filename)

for epoch in range(6):
    if epoch % 3 == 0:
        write_gen_mnist_img(z, vae, epoch)

    train(epoch)

write_gen_mnist_img(z, vae, epoch)


In [ ]:
# for each image in the directory gen_mnist_img, show the image
from IPython.display import Image, display
import matplotlib.pyplot as plt
import os

filenames = sorted(os.listdir('./gen_mnist_img'))

for filename in filenames:
    print("=========================================")
    print(filename)
    display(Image('./gen_mnist_img/' + filename))
    plt.show()


In [ ]:
# Each of the 16 digits is generated by a different 4 dimensional vector. Here
# the variable z is a 16x4 matrix. Each row of z generates a single handwritten digit.
z

In [ ]:
# Let's explore the space of z a little more. First, let's handcraft
# vectors that should exhibit a smooth transition in the output space

z = torch.tensor(
    [
        [ 0.0000,  0.0000,  0.0000,  -1.400],
        [ 0.0000,  0.0000,  0.0000,  -1.200],
        [ 0.0000,  0.0000,  0.0000,  -1.000],
        [ 0.0000,  0.0000,  0.0000,  -0.800],
        [ 0.0000,  0.0000,  0.0000,  -0.600],
        [ 0.0000,  0.0000,  0.0000,  -0.400],
        [ 0.0000,  0.0000,  0.0000,  -0.200],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.2000],
        [ 0.0000,  0.0000,  0.0000,  0.4000],
        [ 0.0000,  0.0000,  0.0000,  0.6000],
        [ 0.0000,  0.0000,  0.0000,  0.8000],
        [ 0.0000,  0.0000,  0.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  1.2000],
        [ 0.0000,  0.0000,  0.0000,  1.4000],
        [ 0.0000,  0.0000,  0.0000,  1.6000],
    ]
)

# Then generate a handwritten digit from that vector.
z_decoded = vae.decoder(z).to(device)

filename = "smooth transtion 1.png"
save_image(z_decoded.view(16, 1, 28, 28), filename)
display(Image(filename))


In [ ]:
# Now try putting in your own numbers below. How many different digits can you 
# smoothly transition between?

z = torch.tensor(
    [
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
    ]
)

# Then generate a handwritten digit from that vector.
z_decoded = vae.decoder(z).to(device)

filename = "smooth transtion 2.png"
save_image(z_decoded.view(16, 1, 28, 28), filename)
display(Image(filename))
